In [25]:

%load_ext autoreload
%autoreload 2

import torch 
import numpy as np 
from dirichlet import * 
import torch
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
input = torch.tensor([[1,2,3],[3,4,6]])

In [30]:
import pyro.distributions as dist 
from pyro.contrib.forecast import eval_crps 

In [29]:
list(range(0,4))
drichilet = dist.Dirichlet(concentration=input)

In [ ]:
eval_crps()

In [ ]:
distributions = Dirichlet(concentration=input)
distributions.concentration

In [ ]:
corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
AREA = 0.5 * 1 * 0.75**0.5
triangle = tri.Triangulation(corners[:, 0], corners[:, 1])

In [ ]:
# For each corner of the triangle, the pair of other corners
pairs = [corners[np.roll(range(3), -i)[1:]] for i in range(3)]
# The area of the triangle formed by point xy and another pair or points
tri_area = lambda xy, pair: 0.5 * np.linalg.norm(np.cross(*(pair - xy)))

def xy2bc(xy, tol=1.e-4):
    '''Converts 2D Cartesian coordinates to barycentric.'''
    coords = np.array([tri_area(xy, p) for p in pairs]) / AREA
    return np.clip(coords, tol, 1.0 - tol)

In [ ]:
class Dirichlet(object):
    def __init__(self, alpha):
        from math import gamma
        from operator import mul
        self._alpha = np.array(alpha)
        self._coef = gamma(np.sum(self._alpha)) / \
                           np.multiply.reduce([gamma(a) for a in self._alpha])
    def pdf(self, x):
        '''Returns pdf value for `x`.'''
        from operator import mul
        return self._coef * np.multiply.reduce([xx ** (aa - 1)
                                               for (xx, aa)in zip(x, self._alpha)])

def draw_pdf_contours(dist, nlevels=200, subdiv=8, **kwargs):
    import math

    refiner = tri.UniformTriRefiner(triangle)
    trimesh = refiner.refine_triangulation(subdiv=subdiv)
    pvals = [dist.pdf(xy2bc(xy)) for xy in zip(trimesh.x, trimesh.y)]

    plt.tricontourf(trimesh, pvals, nlevels, cmap='jet', **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')

In [ ]:
a = torch.tensor([0.9036, 0.9208, 0.9173])
b = torch.tensor([0.7231, 0.1062, 0.1707]) 

diff = a-b 

print(diff)

print(torch.norm(a-b,))
print(torch.norm(diff,))

In [ ]:
np.sqrt(0.1805**2 + 0.8146**2 + 0.7466**2)

In [ ]:
import matplotlib.pyplot as plt 
plt.figure() 
plt.plot([1,2,3], [0,1,2], 'rs', )
plt.plot([1,2,3,4], [0,1,2,3], 'rs', )

plt.figure() 
plt.plot([1,2,3], [0,1,2], 'rs', )
plt.plot([1,2,3,4], [0,1,2,3], 'rs', )



plt.figure() 
plt.plot(1, 2,'rs')

In [ ]:
list(range(0+1))

In [ ]:
output = [0.8077, 0.7874, 0.7990]
target = torch.tensor(
    [0.7231, 0.1062, 0.1707]
)

In [ ]:
output =  torch.tensor(
    
)
print(output)

In [ ]:
distribution = Dirichlet(output)
distribution.rsample()

In [ ]:
distribution.log_prob(target)

In [ ]:
distribution.sample_n(4)

In [ ]:
import math

def lbeta(alpha):
    return sum(math.lgamma(a) for a in alpha) - math.lgamma(sum(alpha))

def ldirichlet_pdf(alpha, theta):
    kernel = sum((a - 1) * math.log(t) for a, t in zip(alpha, theta))
    return kernel - lbeta(alpha)


In [ ]:
ldirichlet_pdf(output, target)

In [ ]:
m = torch.nn.Softmax(dim=1)  
input = torch.randn(2, 3) 
print(input)
print()
output = m(input) 
print(output)
print()
output_exp = torch.exp(output)
print(output_exp)

In [ ]:
draw_pdf_contours(Dirichlet(output_exp[0]))

In [ ]:
torch.clip(torch.tensor([100,200,1]), min = None, max = 100)

In [ ]:
1879%4

In [ ]:
1879//4

In [ ]:
469 * 4 

In [ ]:
468*4 

In [ ]:
469*4